In [ ]:
class Location(object):
    def __init__(self, x, y):
        """x and y are floats"""
        self.x = x
        self.y = y
        
    def move(self, deltaX, deltaY):
        """deltaX and deltaY are floats"""
        return Location(self.x + deltaX, self.y + deltaY)
    
    def getX(self):
        return self.x
    
    def getY(self):
        return self.y
    
    def distFrom(self, other):
        ox = other.x
        oy = other.y
        xDist = self.x - ox
        yDist = self.y - oy
        return (xDist**2 + yDist**2)**0.5   # Pythagorean theorem
    
    def __str__(self):
        return '<' + str(self.x) + ', ' + str(self.y) + '>'
    
    
# Class Field, part 1:

class Field(object):
    def __init__(self):
        self.drunks = {}
        
    def addDrunk(self, drunk, loc):
        if drunk in self.drunks:
            raise ValueError('Duplicate drunk')
        else:
            self.drunks[drunk] = loc
            
    def moveDrunk(self, drunk):
        if drunk not in self.drunks:
            raise ValueError('Drunk not in field')
        xDist, yDist = drunk.takeStep()
        currentLocation = self.drunks[drunk]
        # use move method of location to get new Location
        self.drunks[drunk] = currentLocation.move(xDist, yDist)
       
    def getLoc(self, drunk):
        if drunk not in self.drunks:
            raise ValueError('Drunk not in field')
        return self.drunks[drunk]
    
    
#************* CLASS DRUNK ****************

import random

# A base class to create two more subclass(useful, and cold drunk)

class Drunk(object):
    def __init__(self, name = None):
        """Assumes name is a str"""
        self.name = name
        
    def __str__(self):
        #if self != None:
         #   return self.name
        return 'This drunk is named' + self.name

    
class UsualDrunk(Drunk):
    def takeStep(self):
        stepChoices = [(0.0, 1.0), (0.0, -1.0), (1.0, 0.0), (-1.0, 0.0)]
        return random.choice(stepChoices)
#        return random.choice(stepChoices)
    
class ColdDrunk(Drunk):
    def takeStep(self):
        stepChoices = [(0.0, 0.9), (0.0, -1.1),(1.0, 0.0),(-1.0, 0.0)]
        return random.choice(stepChoices)
# Northward -> less then 1 --> 0.9
# Southward -> more then 1 unit --> 1.1


#************* SIMULATING A SINGLE WALK (Starts here)*********************
#   (I need - class Location, Field and Drunk above this code)

def walk(f, d, numSteps):
    """Assumes: f a Field, d a Drunk in f, and numSteps an int >= 0.
        Moves d numSteps times; Returns the distance between the final
        location and the location at the start of the walk"""
    start = f.getLoc(d)
    for s in range(numSteps):
        f.moveDrunk(d)
    return start.distFrom(f.getLoc(d))

#************* SIMULATING MULTIPLE WALK ***********************

def simWalks(numSteps, numTrials, dClass):
    """Assumes numSteps an int >= 0, numTrials an int > 0,
            dClass a subclass of Drunk
        Simulates numTrials walks of numSteps steps each.
        Returns a list of the final distance for each trial"""

    Homer = dClass()   #creating drunk; dClass is a subclass of class Drunk
    origin = Location(0, 0)
    distances = []
    for t in range(numTrials):
        f = Field()
        f.addDrunk(Homer, origin)
        distances.append(round(walk(f, Homer, numSteps), 1))
    return distances

#********************* PUTTING IT ALL TOGETHER *********************

def drunkTest(walkLengths, numTrials, dClass):
    """Assumes walkLengths a sequence of ints >= 0
        numTrials an int > 0, dClass a subclass or Drun
        For eachnumber of stepls in walkLengths, runs simWalks with
        numTrials walks and prints results"""

    for numSteps in walkLengths:
        distances = simWalks(numSteps, numTrials, dClass)
        print(dClass.__name__, 'random walk of', numSteps, 'steps')
        print('Mean =', round(sum(distances)/len(distances), 4))
        print('Max = ', max(distances), 'Min =', min(distances))
        

        
#random.seed(0)          
#drunkTest((10, 100, 1000), 100, UsualDrunk)


# ******** And the Heat-seeking Drunk ? ****************

#class ColdDrunk(Drunk):
#    def takeStep(self):
#        stepChoices = [(0.0, 0.9),(0.0, -1.1),(1.0, 0.0),(-1.0, 0.0)]
#        return random.choice(stepChoices)
    
def simAll(drunkKinds, walkLengths, numTrials):
    for dClass in drunkKinds:
        drunkTest(walkLengths, numTrials, dClass)
        
        
#random.seed(0)          
#simAll((UsualDrunk, ColdDrunk), (0, 1), 100)

# Iterating over styles

import pylab

class styleIterator(object):
    def __init__(self, styles):
        self.index = 0
        self.styles = styles
    
    def nextStyle(self):
        result = self.styles[self.index]
        if self.index == len(self.styles) -1:
            self.index = 0
        else:
            self.index += 1
        return result
    
def simDrunk(numTrials, dClass, walkLengths):
    meanDistances = []
    for numSteps in walkLengths:
        print('Starting Simulation of', numSteps, 'steps')
        trials = simWalks(numSteps, numTrials, dClass)
        mean = sum(trials)/len(trials)
        meanDistances.append(mean)
    return meanDistances

#simAll(new Version)

def simAll(drunkKinds, walkLengths, numTrials):
    styleChoice = styleIterator(('m-', 'b--', 'g-'))
    for dClass in drunkKinds:
        curStyle = styleChoice.nextStyle()
        print('Starting Simulation of', dClass.__name__)
        means = simDrunk(numTrials, dClass, walkLengths)
        pylab.plot(walkLengths, means, curStyle, 
                   label = dClass.__name__)
        
      
    pylab.title('Mean Distance from Origin (' 
                + str(numTrials) + 'trials)')
    pylab.xlabel('Number of steps')
    pylab.ylabel('Distence from Origin')
    pylab.legend(loc = 'best')
    pylab.semilogx()
    pylab.semilogy()
    pylab.figure('1')
    pylab.clf()


    
#numSteps = (10, 100, 1000, 10000)
#simAll((UsualDrunk, ColdDrunk), numSteps, 100)
#pylab.show()

# Look like Cold Drunk moving so fast, Let's produce a diffent plot

# ************* Getting Ends of Multiple Walks **************

def getFinalLocs(numSteps, numTrials, dClass):
    locs = []
    d = dClass()
    for t in range(numTrials):
        f = Field()
        f.addDrunk(d, Location(0, 0))
        for s in range(numSteps):
            f.moveDrunk(d)
        locs.append(f.getLoc(d))
    return locs

# *************** PLOTTING ENDING LOCATIONS **********

def plotLocs(drunkKinds, numSteps, numTrials):
    styleChoice = styleIterator(('k+', 'r^', 'mo'))
    for dClass in drunkKinds:
        locs = getFinalLocs(numSteps, numTrials, dClass)
        xVals, yVals = [], []
        for loc in locs:
            xVals.append(loc.getX())
            yVals.append(loc.getY())
        meanX = sum(xVals)/len(xVals)
        meanY = sum(yVals)/len(yVals)
        curStyle = styleChoice.nextStyle()
        pylab.plot(xVals, yVals, curStyle,
                  label = dClass.__name__ + 'mean loc. = <'
                  + str(meanX) + ', ' + str(meanY) + '>')
        
        pylab.title('Loction at End of Walks('
                   + str(numSteps) + 'steps')
        pylab.xlabel('Steps East/West of Origin')
        pylab.ylabel('Steps North/South of Origin')
        pylab.legend(loc = 'lower left')
        pylab.figure('1')
        pylab.clf()

        
#random.seed(0)        
#plotLocs((UsualDrunk, ColdDrunk), 100, 200)
#pylab.show()

# OK, Now let's see if I change the properties of the Field  
# Let's add Wormholes ------->

def traceWalk(drunkKinds, numSteps):
    styleChoice = styleIterator(('k+', 'r^', 'mo'))
    f = Field()
    for dClass in drunkKinds:
        d = dClass()
        f.addDrunk(d, Location(0, 0))
        locs = []
        for s in range(numSteps):
            f.moveDrunk(d)
            locs.append(f.getLoc(d))
        xVals, yVals = [], []
        for loc in locs:
            xVals.append(loc.getX())
            yVals.append(loc.getY())
        curStyle = styleChoice.nextStyle()
        pylab.plot(xVals, yVals, curStyle,
                  label = dClass.__name__)
    pylab.title('Spots Visited on Walk('
               + str(numSteps) + 'steps)')
    pylab.xlabel('Steps East/West of Origin')
    pylab.ylabel('Steps North/South of Origin')
    pylab.legend(loc = 'best')
    
    
#traceWalk((UsualDrunk, ColdDrunk), 200)
#pylab.show()

# ********* A SUBCLASS OF FIELD, Part1 *************

class OddField(Field):
    def __init__(self, numHoles, xRange, yRange):
        Field.__init__(self)
        self.wormholes = {}
        for w in range(numHoles):
            x = random.randint(-xRange, xRange)
            y = random.randint(-yRange, yRange)
            newX = random.randint(-xRange, xRange)
            newY = random.randint(-yRange, yRange)
            newLoc = Location(newX, newY)
            self.wormholes[(x, y)] = newLoc
        
    def moveDrunk(self, drunk):
        Field.moveDrunk(self, drunk)
        x = self.drunks[drunk].getX()
        y = self.drunks[drunk].getY()
        if (x, y) in self.wormholes:
            self.drunks[drunk] = self.wormholes[(x, y)]
            


traceWalk((UsualDrunk, ColdDrunk), 500)
pylab.show()